In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

## get Hackernews data

get data from hackernews, save to 
TODO: if works, threads to get 1mil rows

In [2]:
#get data see: https://github.com/HackerNews/API
latestHackerNewsId =  requests.get('https://hacker-news.firebaseio.com/v0/maxitem.json')
print(latestHackerNewsId.json())
latestHackerNewsId = latestHackerNewsId.json()

30125553


In [3]:
latestHackerNews = requests.get('https://hacker-news.firebaseio.com/v0/item/30113196.json')
print(latestHackerNews.json())

{'by': 'dmitriid', 'id': 30113196, 'parent': 30111767, 'text': 'Counter ref: <a href="https:&#x2F;&#x2F;www.quirksmode.org&#x2F;blog&#x2F;archives&#x2F;2021&#x2F;08&#x2F;breaking_the_we.html" rel="nofollow">https:&#x2F;&#x2F;www.quirksmode.org&#x2F;blog&#x2F;archives&#x2F;2021&#x2F;08&#x2F;breaking_th...</a>', 'time': 1643368361, 'type': 'comment'}


In [4]:
def get_hacker_news(id):
    id_as_string = str(id)
    url = f'https://hacker-news.firebaseio.com/v0/item/{id_as_string}.json'
    hacker_news = requests.get(url)    
    return hacker_news.json()

def save_dict_to_csv(dict_to_store, name_of_csv):
    df = pd.DataFrame(dict_to_store)
    df.to_csv(name_of_csv, encoding='utf-8', index=False)

In [5]:
# get hackernews posts
def get_all_hacker_news():
    #number of latest hacker news
    latest_hacker_news = 30113196
    list_of_hacker_dict = []
    
    for x in range(500):
        hacker_dict= {}
        current_json = get_hacker_news(latest_hacker_news)
        for key in current_json:
            hacker_dict[key] = current_json[key]
        list_of_hacker_dict.append(hacker_dict)
        latest_hacker_news -= 1
        
    return list_of_hacker_dict

In [6]:
hackernews_dict_list = get_all_hacker_news()
save_dict_to_csv(hackernews_dict_list, 'hackernews')

## get race data


In [7]:
selected_year = '2021'
url = 'http://ergast.com/api/f1/{year}'.format(year=selected_year)
race_dates_2021 = requests.get(url)

xml_soup = bs(race_dates_2021.content, 'xml')
print(xml_soup)

<?xml version="1.0" encoding="utf-8"?>
<?xml-stylesheet type="text/xsl" href="/schemas/mrd-1.4.xsl"?><MRData limit="30" offset="0" series="f1" total="22" url="http://ergast.com/api/f1/2021" xmlns="http://ergast.com/mrd/1.4">
<RaceTable season="2021">
<Race round="1" season="2021" url="https://en.wikipedia.org/wiki/2021_Bahrain_Grand_Prix">
<RaceName>Bahrain Grand Prix</RaceName>
<Circuit circuitId="bahrain" url="http://en.wikipedia.org/wiki/Bahrain_International_Circuit">
<CircuitName>Bahrain International Circuit</CircuitName>
<Location lat="26.0325" long="50.5106">
<Locality>Sakhir</Locality>
<Country>Bahrain</Country>
</Location>
</Circuit>
<Date>2021-03-28</Date>
<Time>15:00:00Z</Time>
</Race>
<Race round="2" season="2021" url="http://en.wikipedia.org/wiki/2021_Emilia_Romagna_Grand_Prix">
<RaceName>Emilia Romagna Grand Prix</RaceName>
<Circuit circuitId="imola" url="http://en.wikipedia.org/wiki/Autodromo_Enzo_e_Dino_Ferrari">
<CircuitName>Autodromo Enzo e Dino Ferrari</CircuitName>

In [8]:
races = xml_soup.find_all('Race')
names = xml_soup.find_all('RaceName')
countries = xml_soup.find_all('Country')
circuit_names = xml_soup.find_all('CircuitName')
dates = xml_soup.find_all('Date')
times = xml_soup.find_all('Time')

race_dict_list = []

for i in range(0, len(races)):
    race_dict = {}
    race_dict['name'] = names[i].getText()
    race_dict['country'] = countries[i].getText()
    race_dict['circuit_name'] = circuit_names[i].getText()
    race_dict['date'] = dates[i].getText()
    race_dict['time'] = times[i].getText()

    race_dict_list.append(race_dict)

csv_name = 'races_{year}'.format(year=selected_year)
save_dict_to_csv(race_dict_list, 'races')